In [27]:
import xml.etree.ElementTree as ET
import time
import pandas as pd

In [28]:
noparking = ET.parse('Ch_950_Sch_15_ParkingForRestrictedPeriods.xml')
root = noparking.getroot()
head = []

In [29]:
def getvalueofnode(node):
    """ return node text or None """
    return node.text if node is not None else None

In [30]:
dfcols = ['highway', 'side', 'between', 'time','period']
df_xml = pd.DataFrame(columns=dfcols)
for node in root:
    highway = node.find('Highway')
    side = node.find('Side')
    between = node.find('Between')
    time = node.find('Times_and_or_Days')
    period = node.find('Maximum_Period_Permitted')
    
    df_xml = df_xml.append(pd.Series([getvalueofnode(highway), getvalueofnode(side),
                       getvalueofnode(between),getvalueofnode(time),getvalueofnode(period)], index=dfcols),
            ignore_index=True)

In [31]:
df_xml['start_loc'] = df_xml.between.str.split("and").str[0]
df_xml['end_loc'] = df_xml.between.str.split("and").str[1]
df_xml

,highway,side,between,time,period,start_loc,end_loc
0,Tenth Street,Both,Lake Shore Blvd W and the first lane south the...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane south thereof
1,Eleventh Street,Both,Lake Shore Blvd W and the first lane,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane
2,Twelfth Street,None,None,None,None,None,None
3,Thirteenth Street,Both,Lake Shore Boulevard West and the first lane s...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,the first lane south thereof
4,Thirteenth Street,East,Lake Shore Boulevard West and the first lane s...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",None,Lake Shore Boulevard West,the first lane south thereof
5,First Street,East,Lake Shore Boulevard West and a point 26 metre...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,a point 26 metres south
6,Second Street,Both,Lake Shore Blvd W and the first lane south the...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane south thereof
7,Thirty Sixth Street,West,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south
8,Thirty Seventh Street,Both,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south
9,Thirty Seventh Street,West,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south


In [32]:
df = df_xml.dropna()
import csv
df.to_csv("Restricted_Period.csv")

In [ ]:
df = pd.read_csv("No_Parking.csv")
df = df.iloc[:,1:7]
df.shape

In [33]:
import requests
from lxml.html import fromstring
start_lat = []
start_lon=[]

import urllib.request, json 
for index,row in df.iterrows():
    start_st1 = row["highway"].replace(" ","%20")
    start_st2 = row["start_loc"].replace(" ","%20")
    side = row["side"][0]
    if(side == 'B'):
        side = ''
    else:
        side = side+"%20"
    
    url = "http://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates?f=pjson&outFields=Addr_type&forStorage=false&SingleLine="+side+start_st1+"%20and%20"+start_st2+",%20Toronto,%20CA"
    try:
        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
            lon = data["candidates"][0]["location"]["x"]
            lat = data["candidates"][0]["location"]["y"]
    except:
        start_lat.append("")
        start_lon.append("")
        continue
        
    start_lat.append(lat)
    start_lon.append(lon)
    
df['start_lat'] = start_lat
df['start_lon'] = start_lon

C:\Users\fff\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\fff\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [34]:
end_lat = []
end_lon=[]

import urllib.request, json 
for index,row in df.iterrows():
    end_st1 = row["highway"].replace(" ","%20")
    try:
        end_st2 = row["end_loc"].replace(" ","%20")
    except:
        continue
    side = row["side"][0]
    if(side == 'B'):
        side = ''
    else:
        side = side+"%20"
    
    url = "http://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates?f=pjson&outFields=Addr_type&forStorage=false&SingleLine="+side+end_st1+"%20and%20"+end_st2+",%20Toronto,%20CA"
    try:
        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
            lon = data["candidates"][0]["location"]["x"]
            lat = data["candidates"][0]["location"]["y"]
    except:
        end_lat.append("")
        end_lon.append("")
        continue
        
    end_lat.append(lat)
    end_lon.append(lon)

In [35]:
df['end_lat'] = end_lat
df['end_lon'] = end_lon
df

C:\Users\fff\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\fff\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,highway,side,between,time,period,start_loc,end_loc,start_lat,start_lon,end_lat,end_lon
0,Tenth Street,Both,Lake Shore Blvd W and the first lane south the...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane south thereof,43.6,-79.5087,43.5985,-79.508
1,Eleventh Street,Both,Lake Shore Blvd W and the first lane,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane,43.5998,-79.5097,43.5971,-79.5086
3,Thirteenth Street,Both,Lake Shore Boulevard West and the first lane s...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,the first lane south thereof,43.5993,-79.5119,43.6022,-79.513
5,First Street,East,Lake Shore Boulevard West and a point 26 metre...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,a point 26 metres south,43.6023,-79.4985,43.6487,-79.3854
6,Second Street,Both,Lake Shore Blvd W and the first lane south the...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane south thereof,43.602,-79.4997,43.6325,-79.3533
7,Thirty Sixth Street,West,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south,43.5937,-79.537,43.6319,-79.3562
8,Thirty Seventh Street,Both,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south,43.5934,-79.5382,43.5911,-79.5372
9,Thirty Seventh Street,West,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south,43.5934,-79.5382,43.5977,-79.504
10,Third Street,Both,Lake Shore Boulevard West and the first lane s...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,the first lane south thereof,43.6018,-79.5007,43.6326,-79.355
11,Third Street,Both,Lake Shore Boulevard West and the first lane,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,the first lane,43.6018,-79.5007,43.6326,-79.355


In [42]:
df['end_lat'][df['end_lat'] == ''] = None
df['end_lon'][df['end_lon'] == ''] = None

C:\Users\fff\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\fff\Anaconda3\lib\site-packages\pandas\core\generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\fff\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\User

In [43]:
df.end_lat = df.end_lat.astype('float64').fillna(0.0)
df[[ 'end_lat']].iloc[0]
df.end_lon = df.end_lon.astype('float64').fillna(0.0)
df[[ 'end_lon']].iloc[0]

C:\Users\fff\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


end_lon   -79.508025
Name: 0, dtype: float64

In [44]:
df['avg_lat'] = df[['start_lat', 'end_lat']].mean(axis=1)
df['avg_lon'] = df[['start_lon', 'end_lon']].mean(axis=1)
import pygeohash as gh
df['geohash']=df.apply(lambda x: gh.encode(x.avg_lat, x.avg_lon, precision=7), axis=1)

C:\Users\fff\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\fff\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\fff\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata

In [45]:
df

,highway,side,between,time,period,start_loc,end_loc,start_lat,start_lon,end_lat,end_lon,avg_lat,avg_lon,geohash
0,Tenth Street,Both,Lake Shore Blvd W and the first lane south the...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane south thereof,43.6,-79.5087,43.598495,-79.508025,43.598495,-79.508025,dpz2nbb
1,Eleventh Street,Both,Lake Shore Blvd W and the first lane,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane,43.5998,-79.5097,43.597080,-79.508584,43.597080,-79.508584,dpz2n8x
3,Thirteenth Street,Both,Lake Shore Boulevard West and the first lane s...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,the first lane south thereof,43.5993,-79.5119,43.602219,-79.512995,43.602219,-79.512995,dpz2n9s
5,First Street,East,Lake Shore Boulevard West and a point 26 metre...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,a point 26 metres south,43.6023,-79.4985,43.648690,-79.385440,43.648690,-79.385440,dpz83d1
6,Second Street,Both,Lake Shore Blvd W and the first lane south the...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane south thereof,43.602,-79.4997,43.632522,-79.353277,43.632522,-79.353277,dpz84r0
7,Thirty Sixth Street,West,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south,43.5937,-79.537,43.631892,-79.356175,43.631892,-79.356175,dpz84ny
8,Thirty Seventh Street,Both,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south,43.5934,-79.5382,43.591106,-79.537178,43.591106,-79.537178,dpxrypd
9,Thirty Seventh Street,West,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south,43.5934,-79.5382,43.597734,-79.504013,43.597734,-79.504013,dpz2nbd
10,Third Street,Both,Lake Shore Boulevard West and the first lane s...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,the first lane south thereof,43.6018,-79.5007,43.632599,-79.355042,43.632599,-79.355042,dpz84pp
11,Third Street,Both,Lake Shore Boulevard West and the first lane,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,the first lane,43.6018,-79.5007,43.632599,-79.355042,43.632599,-79.355042,dpz84pp


In [46]:
df['time_1'] = df.time.str.split(",").str[0]
df['time_2'] = df.time.str.split(",").str[1]
df

C:\Users\fff\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\fff\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,highway,side,between,time,period,start_loc,end_loc,start_lat,start_lon,end_lat,end_lon,avg_lat,avg_lon,geohash,time_1,time_2
0,Tenth Street,Both,Lake Shore Blvd W and the first lane south the...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane south thereof,43.6,-79.5087,43.598495,-79.508025,43.598495,-79.508025,dpz2nbb,9:00 a.m. to 6:00 p.m.,Mon. to Sat.
1,Eleventh Street,Both,Lake Shore Blvd W and the first lane,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane,43.5998,-79.5097,43.597080,-79.508584,43.597080,-79.508584,dpz2n8x,9:00 a.m. to 6:00 p.m.,Mon. to Sat.
3,Thirteenth Street,Both,Lake Shore Boulevard West and the first lane s...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,the first lane south thereof,43.5993,-79.5119,43.602219,-79.512995,43.602219,-79.512995,dpz2n9s,9:00 a.m. to 6:00 p.m.,Mon. to Sat.
5,First Street,East,Lake Shore Boulevard West and a point 26 metre...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,a point 26 metres south,43.6023,-79.4985,43.648690,-79.385440,43.648690,-79.385440,dpz83d1,9:00 a.m. to 6:00 p.m.,Mon. to Sat.
6,Second Street,Both,Lake Shore Blvd W and the first lane south the...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Blvd W,the first lane south thereof,43.602,-79.4997,43.632522,-79.353277,43.632522,-79.353277,dpz84r0,9:00 a.m. to 6:00 p.m.,Mon. to Sat.
7,Thirty Sixth Street,West,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south,43.5937,-79.537,43.631892,-79.356175,43.631892,-79.356175,dpz84ny,8:00 a.m. to 6:00 p.m.,Mon. to Sat.
8,Thirty Seventh Street,Both,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south,43.5934,-79.5382,43.591106,-79.537178,43.591106,-79.537178,dpxrypd,8:00 a.m. to 6:00 p.m.,Mon. to Sat.
9,Thirty Seventh Street,West,Lake Shore Boulevard West and a point 36.5 met...,"8:00 a.m. to 6:00 p.m., Mon. to Sat.",1 hour,Lake Shore Boulevard West,a point 36.5 metres south,43.5934,-79.5382,43.597734,-79.504013,43.597734,-79.504013,dpz2nbd,8:00 a.m. to 6:00 p.m.,Mon. to Sat.
10,Third Street,Both,Lake Shore Boulevard West and the first lane s...,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,the first lane south thereof,43.6018,-79.5007,43.632599,-79.355042,43.632599,-79.355042,dpz84pp,9:00 a.m. to 6:00 p.m.,Mon. to Sat.
11,Third Street,Both,Lake Shore Boulevard West and the first lane,"9:00 a.m. to 6:00 p.m., Mon. to Sat.",2 hours,Lake Shore Boulevard West,the first lane,43.6018,-79.5007,43.632599,-79.355042,43.632599,-79.355042,dpz84pp,9:00 a.m. to 6:00 p.m.,Mon. to Sat.


In [47]:
df.to_csv("Chapter_950_ParkingForRestrictedPeriods.csv")